In [2]:
import pandas as pd
import numpy as np

In [28]:
import sklearn
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder

# Cargo los CVS 


In [3]:
postulantes_educacion = pd.read_csv('data/datos_navent_fiuba/fiuba_1_postulantes_educacion.csv')
postulantes_educacion = postulantes_educacion.append(pd.read_csv('data/fiuba_hasta_15_abril/fiuba_1_postulantes_educacion.csv'))
postulantes_educacion = postulantes_educacion.append(pd.read_csv('data/fiuba_desde_15_abril/fiuba_1_postulantes_educacion.csv'))

In [4]:
postulantes_genero_y_edad = pd.read_csv('data/datos_navent_fiuba/fiuba_2_postulantes_genero_y_edad.csv')
postulantes_genero_y_edad = postulantes_genero_y_edad.append(pd.read_csv('data/fiuba_hasta_15_abril/fiuba_2_postulantes_genero_y_edad.csv'))
postulantes_genero_y_edad = postulantes_genero_y_edad.append(pd.read_csv('data/fiuba_desde_15_abril/fiuba_2_postulantes_genero_y_edad.csv'))

In [5]:
vistas = pd.read_csv('data/datos_navent_fiuba/fiuba_3_vistas.csv',nrows=10000)
vistas = vistas.append(pd.read_csv('data/fiuba_hasta_15_abril/fiuba_3_vistas.csv',nrows=10000))
vistas = vistas.append(pd.read_csv('data/fiuba_desde_15_abril/fiuba_3_vistas.csv',nrows=10000))

In [6]:
postulaciones = pd.read_csv('data/datos_navent_fiuba/fiuba_4_postulaciones.csv',nrows=10000)
postulaciones = postulaciones.append(pd.read_csv('data/fiuba_hasta_15_abril/fiuba_4_postulaciones.csv',nrows=10000))
# no tiene postulaciones -> pd.read_csv('data/fiuba_desde_15_abril/')

In [7]:
avisos_online = pd.read_csv('data/datos_navent_fiuba/fiuba_5_avisos_online.csv',nrows=10000)
avisos_online = avisos_online.append(pd.read_csv('data/fiuba_hasta_15_abril/fiuba_5_avisos_online.csv',nrows=10000))
# no tiene avisos online pd.read_csv('data/fiuba_desde_15_abril/')

In [8]:
avisos_detalle = pd.read_csv('data/datos_navent_fiuba/fiuba_6_avisos_detalle.csv',nrows=10000)
#avisos_detalle = avisos_detalle.append(pd.read_csv('data/fiuba_hasta_15_abril/fiuba_6_avisos_detalle.csv',nrows=10000))
avisos_detalle = avisos_detalle.append(pd.read_csv('data/fiuba_desde_15_abril/fiuba_6_avisos_detalle.csv',nrows=10000))
avisos_detalle = avisos_detalle.append(pd.read_csv('data/fiuba_6_avisos_detalle_missing_nivel_laboral.csv',nrows=10000))

# Limpio los data frames 

In [9]:
#Elimino duplicados, valores nulos, etc
postulantes_genero_y_edad = postulantes_genero_y_edad[-postulantes_genero_y_edad['fechanacimiento'].isna()]
postulantes_genero_y_edad = postulantes_genero_y_edad.drop_duplicates()
postulantes_genero_y_edad = postulantes_genero_y_edad.drop_duplicates(subset=['idpostulante'],keep='last')

In [10]:
postulantes_educacion  = postulantes_educacion .drop_duplicates(subset=['idpostulante'],keep='last')

In [11]:
avisos_detalle = avisos_detalle.drop_duplicates(subset=['idaviso'],keep='last')

# Juntando datos sobre postulantes

In [12]:
#Uno los data frames de postulantes_genero_y_edad con postulantes_educación para tener todos los datos sobre cada
#postulante y luego hacer un merge con los avisos a los que se han postulado

postulantes = pd.merge(postulantes_educacion,postulantes_genero_y_edad, on ="idpostulante",how="inner")
postulantes.head()

,idpostulante,nombre,estado,fechanacimiento,sexo
0,1d2B,Universitario,En Curso,1976-02-28,MASC
1,6M3jr,Universitario,En Curso,1975-11-03,MASC
2,NrlQx,Terciario/Técnico,En Curso,1973-09-23,FEM
3,Y9RKV,Universitario,En Curso,1982-09-29,MASC
4,ZL608,Universitario,En Curso,1978-04-28,FEM


# Genero DF que informa si el user si postuló o no a un aviso

In [13]:
#Utilizo los df de vistas y postulaciones. Hago un left join de vistas con postulaciones por las columnas idaviso
# y idpostulante. Si estos dos campos aparecen en el df de postulaciones,al hacer el join el campo 
#fecha de postulación podrá contener la fecha o no. Si no la contiene, entonces el usuario solo visitó el aviso.
#Si la contiene, lo visitó y postuló. Teniendo esto en cuenta, modifico el df poniendo 0 (no se postuló) 
# y 1 (se postuló) según corresponda.
vistas = vistas.rename(columns={'idAviso': 'idaviso'})
se_postulo = pd.merge(vistas,postulaciones,how="left",left_on=["idaviso","idpostulante"],right_on=["idaviso","idpostulante"])
se_postulo.head()

,idaviso,timestamp,idpostulante,fechapostulacion
0,1111780242,2018-02-23T13:38:13.187-0500,YjVJQ6Z,NaN
1,1112263876,2018-02-23T13:38:14.296-0500,BmVpYoR,NaN
2,1112327963,2018-02-23T13:38:14.329-0500,wVkBzZd,NaN
3,1112318643,2018-02-23T13:38:17.921-0500,OqmP9pv,NaN
4,1111903673,2018-02-23T13:38:18.973-0500,DrpbXDP,NaN


In [14]:
se_postulo = se_postulo.rename(columns={'fechapostulacion':'se_postulo'})
#por el momento no uso esta columna
se_postulo.drop(columns="timestamp",inplace=True)
#si no tiene fecha, no se postuló, completo con 0
se_postulo['se_postulo']=se_postulo['se_postulo'].fillna(0)
#Los que quedaron con fecha lo reemplazo por 1
se_postulo.loc[se_postulo['se_postulo'] != 0, 'se_postulo'] = 1
#Obtengo el df que me dice qué user se postuló a qué aviso
se_postulo.head()

,idaviso,idpostulante,se_postulo
0,1111780242,YjVJQ6Z,0
1,1112263876,BmVpYoR,0
2,1112327963,wVkBzZd,0
3,1112318643,OqmP9pv,0
4,1111903673,DrpbXDP,0


# Uniendo los datos en un único DF

In [15]:
#Hago join del dataframe anterior con el df que contiene la información sobre los postulantes
train_test_merge = pd.merge(se_postulo, postulantes, on="idpostulante", how="inner")
train_test_merge.head()

,idaviso,idpostulante,se_postulo,nombre,estado,fechanacimiento,sexo
0,1111780242,YjVJQ6Z,0,Universitario,Abandonado,1991-11-16,MASC
1,1112263876,BmVpYoR,0,Universitario,En Curso,1989-12-08,MASC
2,1112356639,BmVpYoR,0,Universitario,En Curso,1989-12-08,MASC
3,1112355153,BmVpYoR,0,Universitario,En Curso,1989-12-08,MASC
4,1112327963,wVkBzZd,0,Universitario,Abandonado,1992-10-06,MASC


In [16]:
#Hago join del dataframe anterior con el df que contiene la información sobre cada aviso
train_test = pd.merge(train_test_merge,avisos_detalle,on="idaviso",how="inner")
train_test.head()

,idaviso,idpostulante,se_postulo,nombre,estado,fechanacimiento,sexo,idpais,titulo,descripcion,nombre_zona,ciudad,mapacalle,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa
0,1111780242,YjVJQ6Z,0,Universitario,Abandonado,1991-11-16,MASC,1,"OPERARIO PARA DEPOSITO, CONFECCION DE PEDIDOS,...",<p>SE NECESITA OPERIARIO PARA DEPOSITO CONFECC...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Programación de producción,dm
1,1111780242,dYjRpMD,0,Secundario,Graduado,1994-04-27,FEM,1,"OPERARIO PARA DEPOSITO, CONFECCION DE PEDIDOS,...",<p>SE NECESITA OPERIARIO PARA DEPOSITO CONFECC...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Programación de producción,dm
2,1111780242,dYjRpMD,0,Secundario,Graduado,1994-04-27,FEM,1,"OPERARIO PARA DEPOSITO, CONFECCION DE PEDIDOS,...",<p>SE NECESITA OPERIARIO PARA DEPOSITO CONFECC...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Programación de producción,dm
3,1111780242,wVopPXl,0,Secundario,Graduado,1969-03-22,MASC,1,"OPERARIO PARA DEPOSITO, CONFECCION DE PEDIDOS,...",<p>SE NECESITA OPERIARIO PARA DEPOSITO CONFECC...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Programación de producción,dm
4,1111780242,wVopPXl,0,Secundario,Graduado,1969-03-22,MASC,1,"OPERARIO PARA DEPOSITO, CONFECCION DE PEDIDOS,...",<p>SE NECESITA OPERIARIO PARA DEPOSITO CONFECC...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Programación de producción,dm


# Transformando los datos

In [18]:
#cambio fecha de nacimiento por la edad
train_test = train_test[(train_test['fechanacimiento'].str.startswith('19'))| (train_test['fechanacimiento'].str.startswith('20'))]
train_test['fechanacimiento'] = pd.to_datetime('today').year-pd.to_datetime(train_test['fechanacimiento']).dt.year
train_test = train_test.rename(columns={'fechanacimiento': 'edad'})
train_test.head()

,idaviso,idpostulante,se_postulo,nombre,estado,edad,sexo,idpais,titulo,descripcion,nombre_zona,ciudad,mapacalle,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa
0,1111780242,YjVJQ6Z,0,Universitario,Abandonado,27,MASC,1,"OPERARIO PARA DEPOSITO, CONFECCION DE PEDIDOS,...",<p>SE NECESITA OPERIARIO PARA DEPOSITO CONFECC...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Programación de producción,dm
1,1111780242,dYjRpMD,0,Secundario,Graduado,24,FEM,1,"OPERARIO PARA DEPOSITO, CONFECCION DE PEDIDOS,...",<p>SE NECESITA OPERIARIO PARA DEPOSITO CONFECC...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Programación de producción,dm
2,1111780242,dYjRpMD,0,Secundario,Graduado,24,FEM,1,"OPERARIO PARA DEPOSITO, CONFECCION DE PEDIDOS,...",<p>SE NECESITA OPERIARIO PARA DEPOSITO CONFECC...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Programación de producción,dm
3,1111780242,wVopPXl,0,Secundario,Graduado,49,MASC,1,"OPERARIO PARA DEPOSITO, CONFECCION DE PEDIDOS,...",<p>SE NECESITA OPERIARIO PARA DEPOSITO CONFECC...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Programación de producción,dm
4,1111780242,wVopPXl,0,Secundario,Graduado,49,MASC,1,"OPERARIO PARA DEPOSITO, CONFECCION DE PEDIDOS,...",<p>SE NECESITA OPERIARIO PARA DEPOSITO CONFECC...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Programación de producción,dm


In [21]:
#Encodeo el sexo 
le = preprocessing.LabelEncoder()
le.fit(train_test['sexo'])
train_test['sexo'] = le.transform(train_test['sexo'])

In [22]:
#POR AHORA drop algunas columnas
train_test.drop(columns = ["idpais","titulo","descripcion","mapacalle"],inplace=True)


In [34]:
#esto no está bueno pero no se me ocurre ahora la mejor manera de trabajar con los campos naN
train_test = train_test.fillna(0)

In [25]:
print(train_test['nivel_laboral'].unique())
print(train_test['tipo_de_trabajo'].unique())
print(train_test['nombre_area'].unique())
print(train_test['denominacion_empresa'].unique())
print(train_test['nombre_zona'].unique())

['Senior / Semi-Senior' 'Junior' 'Otro' 'Jefe / Supervisor / Responsable'
 'Gerencia / Alta Gerencia / Direcci\xc3\xb3n' nan]
['Full-time' 'Part-time' 'Teletrabajo' 'Temporario' 'Por Horas'
 'Por Contrato' 'Pasantia' 'Fines de Semana']
['Programaci\xc3\xb3n de producci\xc3\xb3n' 'Comercial' 'Telemarketing'
 'Multimedia' 'Community Management' 'Administraci\xc3\xb3n'
 'Farmac\xc3\xa9utica' 'Transporte' 'Distribuci\xc3\xb3n'
 'Producci\xc3\xb3n' 'Ventas' 'Call Center' 'Atenci\xc3\xb3n al Cliente'
 'Finanzas' 'Recepcionista' 'Facturaci\xc3\xb3n' 'Compras' 'Salud'
 'Gastronomia' 'Otros' 'Relaciones Institucionales/Publicas' 'Marketing'
 'Contabilidad' 'Seguridad' 'Producto' 'Soporte T\xc3\xa9cnico'
 'Mantenimiento y Limpieza' 'Selecci\xc3\xb3n' 'Impuestos' 'Laboratorio'
 'Calidad' 'Medicina' 'Secretaria' 'Administraci\xc3\xb3n de Personal'
 'Recursos Humanos' 'J\xc3\xb3venes Profesionales' 'Tecnologia / Sistemas'
 'Ventas Internacionales/Exportaci\xc3\xb3n' 'Tesorer\xc3\xada'
 'Dise\xc3\xb

In [44]:
#Para estas columnas no da one hot enconding, por ahora se me ocurrió usar el label encoder
#Pero capaz no esta bueno ponerle el peso que se le está dando con números y es mejor no usarlas directamente
#Por ahora pruebo así
#este capaz da para one hot encoding
le.fit(train_test['nivel_laboral'])
train_test['nivel_laboral'] = le.transform(train_test['nivel_laboral'])
#este capaz da para one hot encoding
le.fit(train_test['tipo_de_trabajo'])
train_test['tipo_de_trabajo'] = le.transform(train_test['tipo_de_trabajo'])
le.fit(train_test['nombre_area'])
train_test['nombre_area'] = le.transform(train_test['nombre_area'])
le.fit(train_test['denominacion_empresa'])
train_test['denominacion_empresa'] = le.transform(train_test['denominacion_empresa'])
le.fit(train_test['nombre_zona'])
#este capaz da para hacer one hot encoding
train_test['nombre_zona'] = le.transform(train_test['nombre_zona'])
#one hot?
le.fit(train_test['nombre'])
train_test['nombre'] = le.transform(train_test['nombre'])
le.fit(train_test['estado'])
train_test['estado'] = le.transform(train_test['estado'])

In [42]:
#Intento one hot encoder
def oneHotEncoding(df, columna):
  
    one_hot = pd.get_dummies(df[columna])
    df.drop(columna, axis=1, inplace=True)
    df = pd.concat([df, one_hot], axis=1)
    return df

train_test = oneHotEncoding(train_test,'nombre_zona')
train_test

KeyError: '[3 3 3 ... 3 3 3] not in index'

In [45]:
train_test.head()

,idaviso,idpostulante,se_postulo,nombre,estado,edad,sexo,nombre_zona,ciudad,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa
0,1111780242,YjVJQ6Z,0,6,0,27,1,3,0,1,5,118,1188
1,1111780242,dYjRpMD,0,4,2,24,0,3,0,1,5,118,1188
2,1111780242,dYjRpMD,0,4,2,24,0,3,0,1,5,118,1188
3,1111780242,wVopPXl,0,4,2,49,1,3,0,1,5,118,1188
4,1111780242,wVopPXl,0,4,2,49,1,3,0,1,5,118,1188


In [53]:
train_test.to_csv(path_or_buf="train.csv",header = ['idaviso', 'idpostulante', 'se_postulo', 'nombre','estado',
       'edad', 'sexo','nombre_zona', 'ciudad', 'tipo_de_trabajo',
       'nivel_laboral', 'nombre_area', 'denominacion_empresa'],index=False)

In [54]:
test = pd.read_csv('train.csv')
test.head()

,idaviso,idpostulante,se_postulo,nombre,estado,edad,sexo,nombre_zona,ciudad,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa
0,1111780242,YjVJQ6Z,0,6,0,27,1,3,0,1,5,118,1188
1,1111780242,dYjRpMD,0,4,2,24,0,3,0,1,5,118,1188
2,1111780242,dYjRpMD,0,4,2,24,0,3,0,1,5,118,1188
3,1111780242,wVopPXl,0,4,2,49,1,3,0,1,5,118,1188
4,1111780242,wVopPXl,0,4,2,49,1,3,0,1,5,118,1188
